In [1]:
import os
import utilities as rocku
import matplotlib.pyplot as plt
import numpy as np
import dxchange
import multiprocessing as mp
import timeit
from tqdm import tqdm
ncore = mp.cpu_count()
print("CPU cores:", ncore)

CPU cores: 64


In [2]:
# need to change
ipts="33087"
# scan_type = 'ct_scans'
scan_type = 'shared/processed_data/normalized'
# scan_type = 'alignment_calibration'

In [3]:
scan_loc = "/HFIR/CG1D/IPTS-"+ipts+"/" + scan_type
scan_list = sorted(os.listdir(scan_loc))
scan_list

['2024_10_25_Empty_cans',
 '2024_10_25_Empty_cans_binned_by_5',
 '2024_10_25_SmF3_0percent',
 '2024_10_25_SmF3_0percent_binned_by_5',
 '2024_10_25_SmF3_10percent',
 '2024_10_25_SmF3_10percent_binned_by_5',
 '2024_10_25_SmF3_1percent',
 '2024_10_25_SmF3_1percent_binned_by_5']

In [4]:
to_rmv = [
'2024_10_25_Empty_cans_binned_by_5',
 '2024_10_25_SmF3_0percent',
 '2024_10_25_SmF3_0percent_binned_by_5',
 '2024_10_25_SmF3_10percent',
 '2024_10_25_SmF3_10percent_binned_by_5',
 '2024_10_25_SmF3_1percent',
 '2024_10_25_SmF3_1percent_binned_by_5'
]
scan_list = rocku.remove_fnames(scan_list, to_rmv)
scan_list

8
1


['2024_10_25_Empty_cans']

# Start the binning

In [9]:
# bin_size = None
bin_size = 5
# bin_method = 'mean'
bin_method = 'median'
# bin_method = 'sum'
##################################################
# assert bin_size > 1
for each_scan in scan_list:
    scan_dir = os.path.join(scan_loc, each_scan)
    fname_list = sorted(os.listdir(scan_dir))
    exposure_list = rocku.get_exposure_list(fname_list)
    name_list = rocku.get_name_list(fname_list)
    # print("Saving to '{}'".format(save_to))
    for each_exposure in exposure_list:
        _fname_list = rocku.filter_list(fname_list, each_exposure)
        #########
        for each_name in name_list:
            __fname_list = rocku.filter_list(_fname_list, each_name)
            if bin_size is None:
                bin_size = len(__fname_list)
                print ("Bin size = {} (No input)".format(bin_size))
            save_to = "/HFIR/CG1D/IPTS-"+ipts+"/shared/processed_data/normalized/" + each_scan + "_binned_by_" + str(bin_size)
            to_rmv = len(__fname_list)%bin_size
            if to_rmv == 0:
                _fname_list_rmv = __fname_list[:]
            else:
                _fname_list_rmv = __fname_list[:-to_rmv]
            if len(_fname_list_rmv) > 0:
                print("Saving to '{}'".format(save_to + "/" + each_name))
            _chunk_list = _fname_list_rmv[0::bin_size]
            for i in tqdm(range(0, len(_chunk_list), 1)):
                _fname_each_chunk = _fname_list_rmv[i:i+bin_size]
                _imgs = rocku.read_tiff_stack_wo_tqdm(scan_dir, _fname_each_chunk)
                if bin_method == 'median':
                    _imgs_bin = np.median(_imgs, axis=0)
                elif bin_method == 'mean':
                    _imgs_bin = np.mean(_imgs, axis=0)
                elif bin_method == 'sum':
                    _imgs_bin = np.sum(_imgs, axis=0)
                # Save files
                _fname = save_to + "/" + _chunk_list[i]
                dxchange.write_tiff(_imgs_bin, fname=_fname, overwrite=True)

Saving to '/HFIR/CG1D/IPTS-33087/shared/processed_data/normalized/2024_10_25_Empty_cans_binned_by_5/empty'


100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.54s/it]
0it [00:00, ?it/s]


In [22]:
bin_size = 5
# bin_method = 'mean'
bin_method = 'median'
##################################################
assert bin_size > 1
for each_scan in scan_list[:2]:
    scan_dir = os.path.join(scan_loc, each_scan)
    fname_list = sorted(os.listdir(scan_dir))
    exposure_list = rocku.get_exposure_list(fname_list)
    name_list = rocku.get_name_list(fname_list)
    save_to = "/HFIR/CG1D/IPTS-"+ipts+"/shared/processed_data/normalized/" + each_scan + "_binned_by_" + str(bin_size)
    print("Saving to '{}'".format(save_to))
    for each_exposure in exposure_list:
        _fname_list = rocku.filter_list(fname_list, each_exposure)
        #########
        for each_name in name_list:
            __fname_list = rocku.filter_list(_fname_list, each_name)
            to_rmv = len(__fname_list)%bin_size
            if to_rmv == 0:
                _fname_list_rmv = __fname_list[:]
            else:
                _fname_list_rmv = __fname_list[:-to_rmv]
            _chunk_list = _fname_list_rmv[0::bin_size]
            for i in tqdm(range(0, len(_chunk_list), 1)):
                _fname_each_chunk = _fname_list_rmv[i:i+bin_size]
                _imgs = rocku.read_tiff_stack_wo_tqdm(scan_dir, _fname_each_chunk)
                if bin_method == 'median':
                    _imgs_bin = np.median(_imgs, axis=0)
                elif bin_method == 'mean':
                    _imgs_bin = np.mean(_imgs, axis=0)
                # Save files
                _fname = save_to + "/" + _chunk_list[i]
                dxchange.write_tiff(_imgs_bin, fname=_fname, overwrite=True)

Saving to '/HFIR/CG1D/IPTS-33087/shared/processed_data/normalized/2024_10_25_SmF3_10percent_binned_by_5'


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

0it [00:00, ?it/s]

# Sort and filter scan files

In [ ]:
###################### need to change start
sample_name = "2024_10_25_SmF3_0percent"
scan_dir = os.path.join(scan_loc, sample_name)
###################### need to change end

In [ ]:
fname_list = os.listdir(scan_dir)
# fname_list.pop(0)
fname_list.sort()
print(len(fname_list))
fname_list

In [ ]:
to_rmv = [
]
fname_list = rocku.remove_fnames(fname_list, to_rmv)

In [ ]:
exposure_list = rocku.get_exposure_list(fname_list)
exposure_list

In [ ]:
name_list = rocku.get_name_list(fname_list)
name_list

In [ ]:
_fname_list = rocku.filter_list(fname_list, name_list[0])
_fname_list